# Class structure

## Import dependences

In [1]:
pip install -U scikit-learn scipy matplotlib

Requirement already up-to-date: scikit-learn in /home/igorfelcam/.local/lib/python3.8/site-packages (1.1.1)
Requirement already up-to-date: scipy in /home/igorfelcam/.local/lib/python3.8/site-packages (1.8.1)
Requirement already up-to-date: matplotlib in /home/igorfelcam/.local/lib/python3.8/site-packages (3.5.2)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /home/igorfelcam/.local/lib/python3.8/site-packages (2.9.1)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip install tensorflow_decision_forests

In [4]:
import random
import string
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import datasets, layers, models, optimizers, utils, losses

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## CnnModel Class

---
Atributes
```
```
---
Methods
```
build(): self
```


In [5]:
class CnnModel:
    @staticmethod

    def build():
        model = models.Sequential()

        model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(10, activation='softmax'))

        model.compile(
            optimizer='adam',
            loss=losses.categorical_crossentropy,
            metrics=['accuracy']
        )

        return model

## RandomForestModel Class

---
Atributes
```
```
---
Methods
```
build(): self
```


In [6]:
class RandomForestModel:
    @staticmethod

    def build():
        return RandomForestClassifier(
            criterion='gini',
            max_depth=10,
            max_features='sqrt',
            min_samples_leaf=10
        )

## KNearestNeighborsModelClass

---
Atributes
```
```
---
Methods
```
build(): self
```


In [7]:
class KNearestNeighborsModel:
    @staticmethod

    def build():
        return KNeighborsClassifier(
            n_neighbors=3,
            weights='distance',
            algorithm='brute'
        )

## DataBatch Class

---
Atributes

```
_test_images: dict = {}
_test_labels: dict = {}
_train_images: dict = {}
_train_labels: dict = {}
_shards: dict = {}
_num_classes = None
_num_clients = None
```

---
Methods
```
getters(): self
_split_data(id_type): void
```

In [8]:
class DataBatch:

    # attributes
    _test_images: dict = {}
    _test_labels: dict = {}
    _train_images: dict = {}
    _train_labels: dict = {}
    _shards: dict = {}
    _num_classes = None
    _num_clients = None


    def __init__(self, num_clients, num_classes):
        self._num_clients = num_clients
        self._num_classes = num_classes


    # getters
    def get_test_images(self, id_type):
        return self._test_images[id_type]


    def get_test_labels(self, id_type):
        return self._test_labels[id_type]


    def get_train_images(self, id_type):
        return self._train_images[id_type]


    def get_train_labels(self, id_type):
        return self._train_labels[id_type]


    def get_shards(self, id_type):
        return self._shards[id_type]


    def get_full_shards(self):
        return self._shards


    # methods
    def _split_data(self, id_type):
        data = list(zip(
            self._train_images[id_type],
            self._train_labels[id_type]
        ))
        random.shuffle(data)
        train, label = zip(*data)

        len_shards = len(self._train_images[id_type]) // self._num_clients
        range_start_shard = 0
        range_final_shard = len_shards

        shards = {}
        i = 0
        for i in range(self._num_clients):
            shards[i] = {
                'train_images': [],
                'train_labels': []
            }

            shards[i]['train_images'] = train[range_start_shard:range_final_shard]
            shards[i]['train_labels'] = label[range_start_shard:range_final_shard]

            range_start_shard = range_final_shard
            range_final_shard = range_final_shard + len_shards

        self._shards[id_type] = shards

## CnnData(DataBatch) Class

---
Atributes

```
__TYPE = 'cnn'
```

---
Methods
```
__prepare_data_to_cnn: void
```

In [9]:
class CnnData(DataBatch):

    # attributes
    __TYPE = 'cnn'


    def __init__(self, dataset, num_clients, num_classes):
        super().__init__(num_clients, num_classes)
        (self._train_images[self.__TYPE], self._train_labels[self.__TYPE]), (self._test_images[self.__TYPE], self._test_labels[self.__TYPE]) = dataset
        self.__prepare_data_to_cnn()
        self._split_data(self.__TYPE)


    # methods
    def __prepare_data_to_cnn(self):
        self._train_labels[self.__TYPE] = utils.to_categorical(
            self._train_labels[self.__TYPE],
            self._num_classes
        )

        self._test_labels[self.__TYPE] = utils.to_categorical(
            self._test_labels[self.__TYPE],
            self._num_classes
        )

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].astype('float32')
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].astype('float32')

        self._train_images[self.__TYPE] /= 255.
        self._test_images[self.__TYPE] /= 255.

## RandomForestData(DataBatch) Class

---
Atributes

```
__TYPE = 'rf'
```

---
Methods
```
__prepare_data_to_rf: void
```

In [10]:
class RandomForestData(DataBatch):

    # attributes
    __TYPE = 'rf'


    def __init__(self, dataset, num_clients, num_classes):
        super().__init__(num_clients, num_classes)
        (self._train_images[self.__TYPE], self._train_labels[self.__TYPE]), (self._test_images[self.__TYPE], self._test_labels[self.__TYPE]) = dataset
        self.__prepare_data_to_rf()
        self._split_data(self.__TYPE)


    # methods
    def __prepare_data_to_rf(self):
        self._train_labels[self.__TYPE] = self._train_labels[self.__TYPE].flatten()
        self._test_labels[self.__TYPE] = self._test_labels[self.__TYPE].flatten()

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].reshape(50000, 3*32*32)
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].reshape(10000, 3*32*32)

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].astype('float32')
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].astype('float32')

        self._train_images[self.__TYPE] /= 255.
        self._test_images[self.__TYPE] /= 255.

## KNearestNeighborsData(DataBatch) Class

---
Atributes

```
__TYPE = 'knn'
```

---
Methods
```
__prepare_data_to_knn: void
```

In [11]:
class KNearestNeighborsData(DataBatch):

    # attributes
    __TYPE = 'knn'


    def __init__(self, dataset, num_clients, num_classes):
        super().__init__(num_clients, num_classes)
        (self._train_images[self.__TYPE], self._train_labels[self.__TYPE]), (self._test_images[self.__TYPE], self._test_labels[self.__TYPE]) = dataset
        self.__prepare_data_to_knn()
        self._split_data(self.__TYPE)


    # methods
    def __prepare_data_to_knn(self):
        self._train_labels[self.__TYPE] = self._train_labels[self.__TYPE].flatten()
        self._test_labels[self.__TYPE] = self._test_labels[self.__TYPE].flatten()

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE]/255.0
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE]/255.0

        n_samples, nx, ny, nrgb = self._train_images[self.__TYPE].shape
        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].reshape((n_samples, nx*ny*nrgb)) # (50000, 32*32*3)

        n_samples, nx, ny, nrgb = self._test_images[self.__TYPE].shape
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].reshape((n_samples, nx*ny*nrgb)) # (10000, 32*32*3)

## ClientHost Class

---
Atributes
```
name: str
data
model
prediction
history
```
---
Methods
```
getters(): self
setters(self): void
fit_model(id_type, test_images, test_labels): void
plot_accuracy(id_type): void
plot_loss(id_type): void
print_details_train(): void
make_predict(id_type, test_sample): void
```


In [12]:
class ClientHost:
    
    # attributes
    __name: str = ''
    __data_batches: dict = {}
    __models: dict = {}
    __histories: dict = {}
    __predictions: dict = {}


    def __init__(self):
        self.__name = ''.join(
            random.choice(
                string.ascii_uppercase + string.digits
            ) for _ in range(6)
        )


    # getters
    @property
    def name(self):
        return self.__name

    @property
    def data_batches(self):
        return self.__data_batches

    @property
    def models(self):
        return self.__models

    @property
    def histories(self):
        return self.__histories
    
    @property
    def predictions(self):
        return self.__predictions


    # setters
    @data_batches.setter
    def data_batches(self, data_batches):
        self.__data_batches = data_batches

    @models.setter
    def models(self, models):
        self.__models = models
    
    @histories.setter
    def histories(self, histories):
        self.__histories = histories

    @predictions.setter
    def predictions(self, predictions):
        self.__predictions = predictions

        
    # class methods
    @staticmethod
    def fit_model(id_type, model, train_images, train_labels, test_images=None, test_labels=None, epochs=None):
        data_batch = {}
        
        if id_type == 'cnn':
            history = model.fit(
                x=tf.stack(train_images),
                y=tf.stack(train_labels),
                epochs=epochs,
                validation_data=(
                    test_images,
                    test_labels
                )
            )
            print(id_type,
                  np.argmax(train_labels[0]),
                  np.argmax(train_labels[2500]),
                  np.argmax(train_labels[4999])
            )
        else:
            history = model.fit(
                X=train_images,
                y=train_labels
            )
            print(id_type, train_labels[0], train_labels[2500], train_labels[4999])
            
        data_batch = {
            'train_images': train_images,
            'train_labels': train_labels
        }
        
        return history, model, data_batch
    

    @staticmethod
    def make_predict(id_type, model, test_sample):
        predictions = []

        for sample in test_sample:
            image = np.expand_dims(sample['image'], 0)
            prediction = model.predict(image)
            
            if id_type == 'cnn':
                predict = np.argmax(prediction[0])
                full_predict = prediction[0]
            else:
                predict = prediction[0]
                full_predict = model.predict_proba(image)

            predictions.append({
                'label': sample['label'],
                'predict': predict,
                'full_predict': full_predict
            })
        
        return predictions
    
    
    # def plot_accuracy(self, id_type):
    #   plt.figure(figsize=[6,4])
    #   plt.plot(self.__histories[id_type].history['accuracy'], 'black', linewidth=2.0)
    #   plt.plot(self.__histories[id_type].history['val_accuracy'], 'blue', linewidth=2.0)
    #   plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
    #   plt.xlabel('Epochs', fontsize=10)
    #   plt.ylabel('Accuracy', fontsize=10)
    #   plt.title('Accuracy Curves', fontsize=12)
    #   plt.show()


    # def plot_loss(self, id_type):
    #   plt.figure(figsize=[6,4])
    #   plt.plot(self.__histories[id_type].history['loss'], 'black', linewidth=2.0)
    #   plt.plot(self.__histories[id_type].history['val_loss'], 'green', linewidth=2.0)
    #   plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
    #   plt.xlabel('Epochs', fontsize=10)
    #   plt.ylabel('Loss', fontsize=10)
    #   plt.title('Loss Curves', fontsize=12)
    #   plt.show()


    # def print_details_train(self):
    #   test_loss, test_acc = self.__models.evaluate(
    #     self._test_images,
    #     self._test_labels,
    #     verbose=2
    #   )

    #   print(
    #     'Client: {} | accuracy: {:.2%} | loss: {} \n\n\n'.format(
    #       self._name,
    #       test_acc,
    #       test_loss
    #     )
    #   )

## ConsensusMechanisms Class

---
Atributes
```
```
---
Methods
```
## public ##
plotHitRate(): void
makeMajority(): void
makeWeightedMajority(): void
makeBordaCount(): void
```


In [22]:
class ConsensusMechanisms:

    # attributes
    # getters
    # setters

    # class methods
    def __is_unanimous(self, voting_list):
        count = 0
        for key, value in voting_list.items():
            if value > 0:
                count = count + 1

        return count == 1
    
    
    def plot_hit_rate(self, voting):
        hits = 0

        for vote in voting:
            if vote['correct']:
                hits += 1

        if hits > 0:
            return 'Hit rate {:.4%}'.format((hits / len(voting)))
        else:
            return 'Hit rate 0%'


    def make_majority(self, type_sample, test_sample, client_list):
        majority_voting = list()

        for label_id, sample in enumerate(test_sample):
            
            if type(sample['label']) is np.ndarray:
                label_value = np.argmax(sample['label'])
            else:
                label_value = sample['label']

            voting = {
                0: 0,
                1: 0,
                2: 0,
                3: 0,
                4: 0,
                5: 0,
                6: 0,
                7: 0,
                8: 0,
                9: 0
            }

            correct = False
            highest_value = 0

            for client in client_list:
                client_prediction = client.predictions[type_sample][label_id]
                voting[client_prediction['predict']] += 1

            highest_value = max(voting, key=voting.get)
            
            if label_value == highest_value:
                correct = True

            unanimous = self.__is_unanimous(voting)

            majority_voting.append(
                {
                    'target': label_value,
                    'correct': correct,
                    'unanimous': unanimous,
                    'voting': voting
                }
            )

        return majority_voting

  
    def make_weighted_majority(self, type_sample, test_sample, client_list):
        weighted_majority_voting = list()
        len_clients = len(client_list)

        for label_id, sample in enumerate(test_sample):
            
            if type(sample['label']) is np.ndarray:
                label_value = np.argmax(sample['label'])
            else:
                label_value = sample['label']

            voting = {
                0: 0,
                1: 0,
                2: 0,
                3: 0,
                4: 0,
                5: 0,
                6: 0,
                7: 0,
                8: 0,
                9: 0
            }
            correct = False

            for client in client_list:
                client_prediction = client.predictions[type_sample][label_id]
                npArgmax = client_prediction['predict']

                if len(client_prediction['full_predict']) == 1:
                    certainty_level = client_prediction['full_predict'][0][npArgmax]
                else:
                    certainty_level = client_prediction['full_predict'][npArgmax]

                certainty_coefficient = certainty_level + 1
                voting[npArgmax] += (certainty_coefficient * certainty_level)

            if label_value == max(voting, key=voting.get):
                correct = True
            
            unanimous = self.__is_unanimous(voting)

            weighted_majority_voting.append(
                {
                    'target': label_value,
                    'correct': correct,
                    'unanimous': unanimous,
                    'voting': voting
                }
            )

        return weighted_majority_voting

  
    def make_borda_count(self, type_sample, test_sample, client_list):
        borda_count_voting = list()
        len_clients = len(client_list)
        clients_ballots = list()

        for client in client_list:
            
            client_ballot = {
                'client': client.name,
                'ballots': list()
            }

            for predictions in client.predictions[type_sample]:
                ballot = list()
                
                if len(predictions['full_predict']) == 1:
                    full_predict = predictions['full_predict'][0]
                else:
                    full_predict = predictions['full_predict']
                    
                for option, confidence in enumerate(full_predict):
                    ballot.append({
                        'option': option,
                        'confidence': confidence
                    })

                ballot.sort(reverse=True, key=lambda e: e['confidence'])
                max_weight = len(ballot)

                for coefficient, vote in enumerate(ballot):
                    vote['weight'] = max_weight - coefficient

                client_ballot['ballots'].append(ballot)

            clients_ballots.append(client_ballot)


        for label_id, sample in enumerate(test_sample):
            
            if type(sample['label']) is np.ndarray:
                label_value = np.argmax(sample['label'])
            else:
                label_value = sample['label']

            voting = {
                0: 0,
                1: 0,
                2: 0,
                3: 0,
                4: 0,
                5: 0,
                6: 0,
                7: 0,
                8: 0,
                9: 0
            }
            correct = False

            for client in clients_ballots:
                for item_ballot in client['ballots'][label_id]:
                    voting[item_ballot['option']] += item_ballot['weight']

            if label_value == max(voting, key=voting.get):
                correct = True
            
            unanimous = self.__is_unanimous(voting)

            borda_count_voting.append({
                'target': label_value,
                'correct': correct,
                'unanimous': unanimous,
                'voting': voting
            })

        return borda_count_voting

# Application excution

## Prepapre dataset

In [14]:
CLIENT_RANGE = 10
CLASSES_RANGE = 10
loaded_data = datasets.cifar10.load_data()


dataset = CnnData(loaded_data, CLIENT_RANGE, CLASSES_RANGE)
dataset = RandomForestData(loaded_data, CLIENT_RANGE, CLASSES_RANGE)
dataset = KNearestNeighborsData(loaded_data, CLIENT_RANGE, CLASSES_RANGE)


print('CNN dataset test len:', len(dataset.get_test_images('cnn')))        # 10.000
print('RF dataset test len:', len(dataset.get_test_images('rf')))          # 10.000
print('KNN dataset test len:', len(dataset.get_test_images('knn')))        # 10.000

print('CNN dataset train len:', len(dataset.get_train_images('cnn')))      # 50.000
print('RF dataset train len:', len(dataset.get_train_images('rf')))        # 50.000
print('KNN dataset train len:', len(dataset.get_train_images('knn')))      # 50.000

print('CNN total clients:', len(dataset.get_shards('cnn')))
print('RF total clients:', len(dataset.get_shards('rf')))
print('KNN total clients:', len(dataset.get_shards('knn')))

CNN dataset test len: 10000
RF dataset test len: 10000
KNN dataset test len: 10000
CNN dataset train len: 50000
RF dataset train len: 50000
KNN dataset train len: 50000
CNN total clients: 10
RF total clients: 10
KNN total clients: 10


## Make sample CNN data

In [15]:
CNN_IMAGES = dataset.get_test_images('cnn')
CNN_LABELS = dataset.get_test_labels('cnn')

cnn_test_sample = list()

SAMPLE_LENGTH = 100   # 100
# SAMPLE_LENGTH = int(len(CNN_IMAGES) * 0.1)   # 10%
# SAMPLE_LENGTH = len(CNN_IMAGES)   # 100%

for i in range(SAMPLE_LENGTH):
    cnn_test_sample.append({
        'image': CNN_IMAGES[i],
        'label': CNN_LABELS[i]
    })

print(SAMPLE_LENGTH, len(cnn_test_sample))

100 100


## Make sample RF data

In [16]:
RF_IMAGES = dataset.get_test_images('rf')
RF_LABELS = dataset.get_test_labels('rf')

rf_test_sample = list()

SAMPLE_LENGTH = 100   # 100
# SAMPLE_LENGTH = int(len(RF_IMAGES) * 0.1)   # 10%
# SAMPLE_LENGTH = len(RF_IMAGES)   # 100%

for i in range(SAMPLE_LENGTH):
    rf_test_sample.append({
        'image': RF_IMAGES[i],
        'label': RF_LABELS[i]
    })

print(SAMPLE_LENGTH, len(rf_test_sample))

100 100


## Make sample KNN data

In [17]:
KNN_IMAGES = dataset.get_test_images('knn')
KNN_LABELS = dataset.get_test_labels('knn')

knn_test_sample = list()

SAMPLE_LENGTH = 100   # 100
# SAMPLE_LENGTH = int(len(KNN_IMAGES) * 0.1)   # 10%
# SAMPLE_LENGTH = len(KNN_IMAGES)   # 100%

for i in range(SAMPLE_LENGTH):
    knn_test_sample.append({
        'image': KNN_IMAGES[i],
        'label': KNN_LABELS[i]
    })

print(SAMPLE_LENGTH, len(knn_test_sample))

100 100


## Prepare clients (federate nodes)

In [18]:
client_list = []

i = 0
for i in range(CLIENT_RANGE):
    client_list.append(ClientHost())


i = 0
for i, client in enumerate(client_list):
    
    histories = {}
    client_models = {}
    data_batches = {}
    predictions = {}
    
    print(i, '-', client.name)

    print('To CNN')
    
    history, model, data_batch = client.fit_model(
        id_type='cnn',
        model=CnnModel().build(),
        train_images=dataset.get_shards('cnn')[i]['train_images'],
        train_labels=dataset.get_shards('cnn')[i]['train_labels'],
        test_images=dataset.get_test_images('cnn'),
        test_labels=dataset.get_test_labels('cnn'),
        epochs=3
    )
    
    predict = client.make_predict(
        id_type='cnn',
        model=model,
        test_sample=cnn_test_sample
    )
    
    histories['cnn'] = history
    client_models['cnn'] = model
    data_batches['cnn'] = data_batch
    predictions['cnn'] = predict
    
    # client.plot_accuracy('cnn')
    # client.plot_loss('cnn')
    print('=====\n')


    print('To Random Forest')
    
    history, model, data_batch = client.fit_model(
        id_type='rf',
        model=RandomForestModel().build(),
        train_images=dataset.get_shards('rf')[i]['train_images'],
        train_labels=dataset.get_shards('rf')[i]['train_labels']
    )
    
    predict = client.make_predict(
        id_type='rf',
        model=model,
        test_sample=rf_test_sample
    )
    
    histories['rf'] = history
    client_models['rf'] = model
    data_batches['rf'] = data_batch
    predictions['rf'] = predict
    
    # client.plot_accuracy('rf')
    # client.plot_loss('rf')
    print('=====\n')
    
    
    print('To K-Nearest Neighbors')
    
    history, model, data_batch = client.fit_model(
        id_type='knn',
        model=KNearestNeighborsModel().build(),
        train_images=dataset.get_shards('knn')[i]['train_images'],
        train_labels=dataset.get_shards('knn')[i]['train_labels']
    )
    
    predict = client.make_predict(
        id_type='knn',
        model=model,
        test_sample=knn_test_sample
    )
    
    histories['knn'] = history
    client_models['knn'] = model
    data_batches['knn'] = data_batch
    predictions['knn'] = predict
    
    # client.plot_accuracy('knn')
    # client.plot_loss('knn')
    
    print('=====\n')
    

    client.histories = histories
    client.models = client_models
    client.data_batches = data_batches
    client.predictions = predictions

0 - MDS5EB
To CNN
Epoch 1/3
157/157 [==============================] - 7s 35ms/step - loss: 2.6309 - accuracy: 0.2200 - val_loss: 3.7331 - val_accuracy: 0.1000
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 1.9966 - accuracy: 0.3226 - val_loss: 3.5292 - val_accuracy: 0.1041
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.8097 - accuracy: 0.3542 - val_loss: 2.8176 - val_accuracy: 0.1818
cnn 2 6 2
1/1 [==============================] - 0s 17ms/step
=====

To Random Forest
rf 4 4 0
=====

To K-Nearest Neighbors
knn 5 8 9
=====

1 - GJ9CN2
To CNN
Epoch 1/3
157/157 [==============================] - 6s 31ms/step - loss: 2.6802 - accuracy: 0.2226 - val_loss: 4.5586 - val_accuracy: 0.1000
Epoch 2/3
157/157 [==============================] - 5s 33ms/step - loss: 2.0758 - accuracy: 0.3040 - val_loss: 2.9951 - val_accuracy: 0.1753
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.7729 - accuracy: 0.3738 - val_loss: 

1/1 [==============================] - 0s 19ms/step
=====

To Random Forest
rf 2 9 8
=====

To K-Nearest Neighbors
knn 3 8 1
=====

2 - I3LGBW
To CNN
Epoch 1/3
157/157 [==============================] - 6s 32ms/step - loss: 2.5871 - accuracy: 0.2234 - val_loss: 3.6504 - val_accuracy: 0.1000
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 1.9673 - accuracy: 0.3382 - val_loss: 3.8214 - val_accuracy: 0.1041
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.7669 - accuracy: 0.3836 - val_loss: 2.0539 - val_accuracy: 0.2740
cnn 0 4 4
1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 19ms/step
=====

To Random Forest
rf 2 8 1
=====

To K-Nearest Neighbors
knn 3 4 3
=====

3 - ZVPFHF
To CNN
Epoch 1/3
157/157 [==============================] - 6s 32ms/step - loss: 2.5279 - accuracy: 0.2356 - val_loss: 2.8970 - val_accuracy: 0.1082
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 1.9644 - accuracy: 0.3360 - val_loss: 2.8079 - val_accuracy: 0.1504
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.7504 - accuracy: 0.3818 - val_loss: 2.4213 - val_accuracy: 0.2359
cnn 2 0 7
1/1 [==============================] - 0s 21ms/step
=====

To Random Forest
rf 1 1 2
=====

To K-Nearest Neighbors
knn 0 0 2
=====

4 - 69BI8C
To CNN
Epoch 1/3
157/157 [==============================] - 6s 32ms/step - loss: 2.5325 - accuracy: 0.2326 - val_loss: 4.3567 - val_accuracy: 0.1086
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 2.0020 - accuracy: 0.3144 - val_loss: 3.4228 - v

1/1 [==============================] - 0s 20ms/step
=====

To Random Forest
rf 9 8 9
=====

To K-Nearest Neighbors
knn 3 0 3
=====

5 - ANIU0V
To CNN
Epoch 1/3
157/157 [==============================] - 6s 32ms/step - loss: 2.6016 - accuracy: 0.2332 - val_loss: 3.7694 - val_accuracy: 0.1000
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 2.0062 - accuracy: 0.3166 - val_loss: 4.1141 - val_accuracy: 0.1108
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.7872 - accuracy: 0.3604 - val_loss: 3.0052 - val_accuracy: 0.1702
cnn 0 4 4
1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step
=====

To Random Forest
rf 1 2 5
=====

To K-Nearest Neighbors
knn 5 7 3
=====

6 - YQK7X9
To CNN
Epoch 1/3
157/157 [==============================] - 6s 32ms/step - loss: 2.6167 - accuracy: 0.2278 - val_loss: 3.4423 - val_accuracy: 0.1169
Epoch 2/3
157/157 [==============================] - 4s 29ms/step - loss: 2.0268 - accuracy: 0.3190 - val_loss: 3.1431 - val_accuracy: 0.1238
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.7853 - accuracy: 0.3702 - val_loss: 2.3037 - val_accuracy: 0.2487
cnn 5 1 8
1/1 [==============================] - 0s 19ms/step
=====

To Random Forest
rf 6 1 6
=====

To K-Nearest Neighbors
knn 6 8 4
=====

7 - K33FI5
To CNN
Epoch 1/3
157/157 [==============================] - 6s 31ms/step - loss: 2.5808 - accuracy: 0.2444 - val_loss: 3.5312 - val_accuracy: 0.1095
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 1.9904 - accuracy: 0.3234 - val_loss: 3.0834 - v

1/1 [==============================] - 0s 20ms/step
=====

To Random Forest
rf 6 9 7
=====

To K-Nearest Neighbors
knn 3 6 9
=====

8 - UT1H9X
To CNN
Epoch 1/3
157/157 [==============================] - 6s 32ms/step - loss: 2.5948 - accuracy: 0.2222 - val_loss: 3.1865 - val_accuracy: 0.1000
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 2.0049 - accuracy: 0.3180 - val_loss: 3.2027 - val_accuracy: 0.1328
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.7980 - accuracy: 0.3670 - val_loss: 2.3991 - val_accuracy: 0.1949
cnn 3 9 3
1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step
=====

To Random Forest
rf 2 4 5
=====

To K-Nearest Neighbors
knn 7 2 1
=====

9 - 7XNYFS
To CNN
Epoch 1/3
157/157 [==============================] - 6s 32ms/step - loss: 2.5798 - accuracy: 0.2286 - val_loss: 3.0712 - val_accuracy: 0.1026
Epoch 2/3
157/157 [==============================] - 4s 28ms/step - loss: 2.0168 - accuracy: 0.3106 - val_loss: 3.9442 - val_accuracy: 0.1117
Epoch 3/3
157/157 [==============================] - 4s 28ms/step - loss: 1.8216 - accuracy: 0.3502 - val_loss: 2.2507 - val_accuracy: 0.2246
cnn 8 3 1
1/1 [==============================] - 0s 20ms/step
=====

To Random Forest


rf 6 8 3
=====

To K-Nearest Neighbors
knn 4 0 8
=====



## Execute

In [23]:
consensus_mechanisms = ConsensusMechanisms()

types_sample = [
    'cnn',
    'rf',
    'knn'
]

majority_voting = {}
weighted_majority_voting = {}
borda_count_voting = {}

for type_sample in types_sample:
    print('To', type_sample)

    # --
    majority_voting[type_sample] = consensus_mechanisms.make_majority(
        type_sample,
        globals().get(type_sample + '%s' % '_test_sample'),
        client_list
    )
    print('Majority', consensus_mechanisms.plot_hit_rate(majority_voting[type_sample]))

    # --
    weighted_majority_voting[type_sample] = consensus_mechanisms.make_weighted_majority(
        type_sample,
        globals().get(type_sample + '%s' % '_test_sample'),
        client_list
    )
    print('Weighted Majority', consensus_mechanisms.plot_hit_rate(weighted_majority_voting[type_sample]))

    # --
    borda_count_voting[type_sample] = consensus_mechanisms.make_borda_count(
        type_sample,
        globals().get(type_sample + '%s' % '_test_sample'),
        client_list
    )
    print('Borda Count', consensus_mechanisms.plot_hit_rate(borda_count_voting[type_sample]))

    print('\n')



for i in random.sample(range(100), k=5):
    print('Index sample', i)
    print('\n')
    
    print('To cnn')
    print('Majority', majority_voting['cnn'][i])
    print('Weighted Majority', weighted_majority_voting['cnn'][i])
    print('Borda Count', borda_count_voting['cnn'][i])
    print('\n')

    print('To rf')
    print('Majority', majority_voting['rf'][i])
    print('Weighted Majority', weighted_majority_voting['rf'][i])
    print('Borda Count', borda_count_voting['rf'][i])
    print('\n')
    
    print('To knn')
    print('Majority', majority_voting['knn'][i])
    print('Weighted Majority', weighted_majority_voting['knn'][i])
    print('Borda Count', borda_count_voting['knn'][i])
    print('\n')

To cnn
Majority Hit rate 18.0000%
Weighted Majority Hit rate 23.0000%
Borda Count Hit rate 19.0000%


To rf
Majority Hit rate 44.0000%
Weighted Majority Hit rate 44.0000%
Borda Count Hit rate 44.0000%


To knn
Majority Hit rate 29.0000%
Weighted Majority Hit rate 32.0000%
Borda Count Hit rate 31.0000%


Index sample 89


To cnn
Majority {'target': 9, 'correct': False, 'unanimous': False, 'voting': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 3, 7: 0, 8: 7, 9: 0}}
Weighted Majority {'target': 9, 'correct': False, 'unanimous': False, 'voting': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 2.1586934278445797, 7: 0, 8: 4.365000053881562, 9: 0}}
Borda Count {'target': 9, 'correct': False, 'unanimous': False, 'voting': {0: 68, 1: 47, 2: 65, 3: 35, 4: 65, 5: 29, 6: 65, 7: 24, 8: 87, 9: 65}}


To rf
Majority {'target': 9, 'correct': True, 'unanimous': False, 'voting': {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 3, 9: 6}}
Weighted Majority {'target': 9, 'correct': True, 'unanimous': False, 'voting

In [ ]:
# # DEBUG TEST

# label = 2

# import pprint
# pp = pprint.PrettyPrinter(indent=2)


# print('cnn')
# test = client_list[0].get_predictions_by_type('cnn')

# # print(test[label]['full_predict'])

# ballot = list()
# for option, confidence in enumerate(test[label]['full_predict']):
#     ballot.append({
#         'option': option,
#         'confidence': confidence
#     })

# ballot.sort(reverse=True, key=lambda e: e['confidence'])

# for coefficient, vote in enumerate(ballot):
#     vote['weight'] = len(ballot) - coefficient

# pp.pprint(ballot)



# print('rf')
# test = client_list[0].get_predictions_by_type('rf')

# # print(test[label]['full_predict'][0])

# ballot = list()
# for option, confidence in enumerate(test[label]['full_predict'][0]):
#     ballot.append({
#         'option': option,
#         'confidence': confidence
#     })

# ballot.sort(reverse=True, key=lambda e: e['confidence'])

# for coefficient, vote in enumerate(ballot):
#     vote['weight'] = len(ballot) - coefficient

# pp.pprint(ballot)



# print('knn')
# test = client_list[0].get_predictions_by_type('knn')

# # print(test[label]['full_predict'][0])

# ballot = list()
# for option, confidence in enumerate(test[label]['full_predict'][0]):
#     ballot.append({
#         'option': option,
#         'confidence': confidence
#     })

# ballot.sort(reverse=True, key=lambda e: e['confidence'])

# for coefficient, vote in enumerate(ballot):
#     vote['weight'] = len(ballot) - coefficient

# pp.pprint(ballot)

In [24]:
types_sample = [
    'cnn',
    'rf',
    'knn'
]

for client in client_list:
    print('Client: ', client.name)
    
    for type_sample in types_sample:
        print(client.models[type_sample])
        
        if type_sample != 'cnn':
            print(
                'Score model: {:.2%}'.format(
                    client.models[type_sample].score(
                        dataset.get_test_images(type_sample),
                        dataset.get_test_labels(type_sample)
                    )
                )
            )
        else:
            print(
                'Score model: {:.2%}'.format(
                    client.models[type_sample].evaluate(
                        dataset.get_test_images(type_sample),
                        dataset.get_test_labels(type_sample),
                        verbose=0
                    )[1]
                )
            )
            
        
    print('\n') 

Client:  MDS5EB
Score model: 18.18%
RandomForestClassifier(max_depth=10, min_samples_leaf=10)
Score model: 38.65%
KNeighborsClassifier(algorithm='brute', n_neighbors=3, weights='distance')
Score model: 27.85%


Client:  GJ9CN2
Score model: 20.05%
RandomForestClassifier(max_depth=10, min_samples_leaf=10)
Score model: 38.84%
KNeighborsClassifier(algorithm='brute', n_neighbors=3, weights='distance')
Score model: 28.63%


Client:  I3LGBW
Score model: 27.40%
RandomForestClassifier(max_depth=10, min_samples_leaf=10)
Score model: 38.58%
KNeighborsClassifier(algorithm='brute', n_neighbors=3, weights='distance')
Score model: 28.31%


Client:  ZVPFHF
Score model: 23.59%
RandomForestClassifier(max_depth=10, min_samples_leaf=10)
Score model: 39.33%
KNeighborsClassifier(algorithm='brute', n_neighbors=3, weights='distance')
Score model: 28.21%


Client:  69BI8C
Score model: 20.15%
RandomForestClassifier(max_depth=10, min_samples_leaf=10)
Score model: 39.19%
KNeighborsClassifier(algorithm='brute', n_

In [27]:
for client in client_list:
    print('Client: ', client.name)

    print(
        'cnn',
        np.argmax(client.data_batches['cnn']['train_labels'][0]),
        np.argmax(client.data_batches['cnn']['train_labels'][2500]),
        np.argmax(client.data_batches['cnn']['train_labels'][4999]),
        '\n'
    )

Client:  MDS5EB
cnn 2 6 2 

Client:  GJ9CN2
cnn 3 1 8 

Client:  I3LGBW
cnn 0 4 4 

Client:  ZVPFHF
cnn 2 0 7 

Client:  69BI8C
cnn 6 2 3 

Client:  ANIU0V
cnn 0 4 4 

Client:  YQK7X9
cnn 5 1 8 

Client:  K33FI5
cnn 6 4 9 

Client:  UT1H9X
cnn 3 9 3 

Client:  7XNYFS
cnn 8 3 1 



In [26]:
i = 0
for i, client in enumerate(client_list):

    print(
        'cnn',
        np.argmax(dataset.get_shards('cnn')[i]['train_labels'][0]),
        np.argmax(dataset.get_shards('cnn')[i]['train_labels'][2500]),
        np.argmax(dataset.get_shards('cnn')[i]['train_labels'][4999])
    )
    
    print(
        'rf',
        dataset.get_shards('rf')[i]['train_labels'][0],
        dataset.get_shards('rf')[i]['train_labels'][2500],
        dataset.get_shards('rf')[i]['train_labels'][4999]
    )
        
    print(
        'knn',
        dataset.get_shards('knn')[i]['train_labels'][0],
        dataset.get_shards('knn')[i]['train_labels'][2500],
        dataset.get_shards('knn')[i]['train_labels'][4999],
        '\n'
    )

cnn 2 6 2
rf 4 4 0
knn 5 8 9 

cnn 3 1 8
rf 2 9 8
knn 3 8 1 

cnn 0 4 4
rf 2 8 1
knn 3 4 3 

cnn 2 0 7
rf 1 1 2
knn 0 0 2 

cnn 6 2 3
rf 9 8 9
knn 3 0 3 

cnn 0 4 4
rf 1 2 5
knn 5 7 3 

cnn 5 1 8
rf 6 1 6
knn 6 8 4 

cnn 6 4 9
rf 6 9 7
knn 3 6 9 

cnn 3 9 3
rf 2 4 5
knn 7 2 1 

cnn 8 3 1
rf 6 8 3
knn 4 0 8 

